### Movie Agent

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

### Import ADK components

In [ ]:
import json
import requests
import subprocess
import time
import uuid

from google.adk.agents import LlmAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)

from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Hide additional warnings in the notebook
import warnings

warnings.filterwarnings("ignore")

print("✅ ADK components imported successfully.")

### Configure Retry Options

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## 📦 Section 1: Create the Movie Catalog Agent (To Be Exposed)

We'll create a **Movie  Catalog Agent** that provides movie information from an external vendor's catalog. This agent will be **exposed via A2A** so other agents (like customer support) can use it.

In [ ]:
# Define a movie catalog lookup tool
# In a real system, this would query the vendor's movie database
def get_movie_info(movie_name: str) -> str:
    """Get movie information for a given movie.

    Args:
        movie_name: Name of the movie (e.g., "iPhone 15 Pro", "MacBook Pro")

    Returns:
        movie information as a string
    """
    # Mock movie catalog - in production, this would query a real database
    movie_catalog = {
        "on the waterfront": "Marlon Brando, A washed-up boxer turned longshoreman finds his conscience and risks everything to stand up against the corrupt mob-controlled union on the docks.",
        "the martian": "Matt Damon, Stranded alone on Mars after his crew thinks he’s dead, astronaut Mark Watney uses ingenuity, sarcasm, and potatoes to survive until Earth can bring him home.",
        "inception": "Leonardo DiCaprio, A professional thief who steals secrets from dreams is offered one last impossible job: to plant an idea inside a billionaire heir’s mind instead of stealing one.",
        "dil chahta hai": 'Aamir Khan, Three inseparable college friends discover that growing up means falling in love, drifting apart, and eventually finding their way back to each other.',
        "sholay": 'Amitabh Bachchan, A ruthless dacoit terrorizes a village until a grizzled ex-cop hires two small-time crooks—Jai and Veeru—to hunt him down in an epic tale of revenge and bromance.',
        "lillies of the field": 'Sydney Poitier, A wandering handyman with a free spirit reluctantly helps a group of German nuns build a chapel in the Arizona, finding purpose along the way.',
        "one flew over the cuckoo's nest": 'Jack Nicholson, Rebellious con-man Randle P. McMurphy fakes insanity to serve his sentence in a mental ward, only to ignite a war of wills against the tyrannical Nurse Ratched.',
    }

    movie_lower = movie_name.lower().strip()

    if movie_lower in movie_catalog:
        return f"Movie: {movie_catalog[movie_lower]}"
    else:
        available = ", ".join([p.title() for p in movie_catalog.keys()])
        return f"Sorry, I don't have information for {movie_name}. Available movies: {available}"


# Create the movie Catalog Agent
# This agent specializes in providing movie information from the vendor's catalog
movie_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="movie_catalog_agent",
    description="External vendor's movie catalog agent that provides movie information and availability.",
    instruction="""
    You are a movie catalog specialist from an external vendor.
    When asked about movies, use the get_movie_info tool to fetch data from the catalog.
    Provide clear, accurate movie information.
    If asked about multiple movies, look up each one.
    Be professional and helpful.
    """,
    tools=[get_movie_info],  # Register the movie lookup tool
)

print("✅ Movie Catalog Agent created successfully!")
print("   Model: gemini-2.5-flash-lite")
print("   Tool: get_movie_info()")
print("   Ready to be exposed via A2A...")

## 🌐 Section 2: Expose the Movie Catalog Agent via A2A

Now we'll use ADK's `to_a2a()` function to make our Movie Catalog Agent accessible to other agents.

### What `to_a2a()` does:
- 🔧 Wraps your agent in an A2A-compatible server (FastAPI/Starlette)
- 📋 Auto-generates an **agent card** that includes:
  - Agent name, description, and version
  - Skills (your tools/functions become "skills" in A2A)
  - Protocol version and endpoints
  - Input/output modes
- 🌐 Serves the agent card at `/.well-known/agent-card.json` (standard A2A path)
- ✨ Handles all A2A protocol details (request/response formatting, task endpoints)

This is the **easiest way** to expose an ADK agent via A2A!

**💡 Key Concept: Agent Cards**

An **agent card** is a JSON document that serves as a "business card" for your agent. It describes:
- What the agent does (name, description, version)
- What capabilities it has (skills, tools, functions)  
- How to communicate with it (URL, protocol version, endpoints)

Every A2A agent must publish its agent card at the standard path: `/.well-known/agent-card.json`

Think of it as the "contract" that tells other agents how to work with your agent.

📖 **Learn more:**
- [Exposing Agents with ADK](https://google.github.io/adk-docs/a2a/quickstart-exposing/)
- [A2A Protocol Specification](https://a2a-protocol.org/latest/specification/)

In [ ]:
# Convert the movie catalog agent to an A2A-compatible application
# This creates a FastAPI/Starlette app that:
#   1. Serves the agent at the A2A protocol endpoints
#   2. Provides an auto-generated agent card
#   3. Handles A2A communication protocol
movie_catalog_a2a_app = to_a2a(
    movie_catalog_agent, port=8001  # Port where this agent will be served
)

print("✅ Movie Catalog Agent is now A2A-compatible!")
print("   Agent will be served at: http://localhost:8001")
print("   Agent card will be at: http://localhost:8001/.well-known/agent-card.json")
print("   Ready to start the server...")

## 🚀 Section 3: Start the Movie Catalog Agent Server

We'll start the Movie Catalog Agent server in the **background** using `uvicorn`, so it can serve requests from other agents.

### Why run in background?
- The server needs to keep running while we create and test the Customer Support Agent
- This simulates a real-world scenario where different agents run as separate services
- In production, the vendor would host this on their infrastructure

In [ ]:
# First, let's save the movie catalog agent to a file that uvicorn can import
movie_catalog_agent_code = '''
import os
from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

def get_movie_info(movie_name: str) -> str:
    """Get movie information for a given movie."""
    movie_catalog = {
        "on the waterfront": "Marlon Brando, A washed-up boxer turned longshoreman finds his conscience and risks everything to stand up against the corrupt mob-controlled union on the docks.",
        "the martian": "Matt Damon, Stranded alone on Mars after his crew thinks he’s dead, astronaut Mark Watney uses ingenuity, sarcasm, and potatoes to survive until Earth can bring him home.",
        "inception": "Leonardo DiCaprio, A professional thief who steals secrets from dreams is offered one last impossible job: to plant an idea inside a billionaire heir’s mind instead of stealing one.",
        "dil chahta hai": 'Aamir Khan, Three inseparable college friends discover that growing up means falling in love, drifting apart, and eventually finding their way back to each other.',
        "sholay": 'Amitabh Bachchan, A ruthless dacoit terrorizes a village until a grizzled ex-cop hires two small-time crooks—Jai and Veeru—to hunt him down in an epic tale of revenge and bromance.',
        "lillies of the field": 'Sydney Poitier, A wandering handyman with a free spirit reluctantly helps a group of German nuns build a chapel in the Arizona, finding purpose along the way.',
        "one flew over the cuckoo's nest": 'Jack Nicholson, Rebellious con-man Randle P. McMurphy fakes insanity to serve his sentence in a mental ward, only to ignite a war of wills against the tyrannical Nurse Ratched.',
    }
    
    movie_lower = movie_name.lower().strip()
    
    if movie_lower in movie_catalog:
        return f"Movie: {movie_catalog[movie_lower]}"
    else:
        available = ", ".join([p.title() for p in movie_catalog.keys()])
        return f"Sorry, I don't have information for {movie_name}. Available movies: {available}"

movie_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="movie_catalog_agent",
    description="External vendor's movie catalog agent that provides movie information and availability.",
    instruction="""
    You are a movie catalog specialist from an external vendor.
    When asked about movies, use the get_movie_info tool to fetch data from the catalog.
    Provide clear, accurate movie information including price, availability, and specs.
    If asked about multiple movies, look up each one.
    Be professional and helpful.
    """,
    tools=[get_movie_info]
)

# Create the A2A app
app = to_a2a(movie_catalog_agent, port=8001)
'''

# Write the movie catalog agent to a temporary file
with open("/tmp/movie_catalog_server.py", "w") as f:
    f.write(movie_catalog_agent_code)

print("📝 movie Catalog agent code saved to /tmp/movie_catalog_server.py")

# Start uvicorn server in background
# Note: We redirect output to avoid cluttering the notebook
server_process = subprocess.Popen(
    [
        "uvicorn",
        "movie_catalog_server:app",  # Module:app format
        "--host",
        "localhost",
        "--port",
        "8001",
    ],
    cwd="/tmp",  # Run from /tmp where the file is
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ},  # Pass environment variables (including GOOGLE_API_KEY)
)

print("🚀 Starting Movie Catalog Agent server...")
print("   Waiting for server to be ready...")

# Wait for server to start (poll until it responds)
max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = requests.get(
            "http://localhost:8001/.well-known/agent-card.json", timeout=1
        )
        if response.status_code == 200:
            print(f"\n✅ Movie Catalog Agent server is running!")
            print(f"   Server URL: http://localhost:8001")
            print(f"   Agent card: http://localhost:8001/.well-known/agent-card.json")
            break
    except requests.exceptions.RequestException:
        time.sleep(5)
        print(".", end="", flush=True)
else:
    print("\n⚠️  Server may not be ready yet. Check manually if needed.")

# Store the process so we can stop it later
globals()["movie_catalog_server_process"] = server_process

### 🔍 View the Auto-Generated Agent Card

The `to_a2a()` function automatically created an **agent card** that describes the Movie Catalog Agent's capabilities. Let's take a look!

In [ ]:
# Fetch the agent card from the running server
try:
    response = requests.get(
        "http://localhost:8001/.well-known/agent-card.json", timeout=5
    )

    if response.status_code == 200:
        agent_card = response.json()
        print("📋 Movie Catalog Agent Card:")
        print(json.dumps(agent_card, indent=2))

        print("\n✨ Key Information:")
        print(f"   Name: {agent_card.get('name')}")
        print(f"   Description: {agent_card.get('description')}")
        print(f"   URL: {agent_card.get('url')}")
        print(f"   Skills: {len(agent_card.get('skills', []))} capabilities exposed")
    else:
        print(f"❌ Failed to fetch agent card: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching agent card: {e}")
    print("   Make sure the movie Catalog Agent server is running (previous cell)")

## 🎧 Section 4: Create the Customer Support Agent (Consumer)

Now we'll create a **Customer Support Agent** that consumes the Movie Catalog Agent using A2A.

### How it works:
1. We use `RemoteA2aAgent` to create a **client-side proxy** for the Movie Catalog Agent
2. The Customer Support Agent can use the Movie Catalog Agent like any other tool
3. ADK handles all the A2A protocol communication behind the scenes

This demonstrates the power of A2A: **agents can collaborate as if they were local!**

**How RemoteA2aAgent works:**
- It's a **client-side proxy** that reads the remote agent's card
- Translates sub-agent calls into A2A protocol requests (HTTP POST to `/tasks`)
- Handles all the protocol details so you just use it like a regular sub-agent

📖 **Learn more:**
- [Consuming Remote Agents with ADK](https://google.github.io/adk-docs/a2a/quickstart-consuming/)
- [What is A2A?](https://a2a-protocol.org/latest/topics/what-is-a2a/)

In [ ]:
# Create a RemoteA2aAgent that connects to our Movie Catalog Agent
# This acts as a client-side proxy - the Customer Support Agent can use it like a local agent
remote_movie_catalog_agent = RemoteA2aAgent(
    name="movie_catalog_agent",
    description="Remote movie catalog agent from external vendor that provides movie information.",
    # Point to the agent card URL - this is where the A2A protocol metadata lives
    agent_card=f"http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("✅ Remote Movie Catalog Agent proxy created!")
print(f"   Connected to: http://localhost:8001")
print(f"   Agent card: http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}")
print("   The Customer Support Agent can now use this like a local sub-agent!")

In [ ]:
# Now create the Customer Support Agent that uses the remote Movie Catalog Agent
customer_support_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="customer_support_agent",
    description="A customer support assistant that helps customers with movie inquiries and information.",
    instruction="""
    You are a friendly and professional customer support agent.
    
    When customers ask about movies:
    1. Use the movie_catalog_agent sub-agent to look up movie information
    2. Provide clear answers about the movie information 
    3. If a movie is out of stock, mention the expected availability
    4. Be helpful and professional!
    
    Always get movie information from the movie_catalog_agent before answering customer questions.
    """,
    sub_agents=[remote_movie_catalog_agent],  # Add the remote agent as a sub-agent!
)

print("✅ Customer Support Agent created!")
print("   Model: gemini-2.5-flash-lite")
print("   Sub-agents: 1 (remote Movie Catalog Agent via A2A)")
print("   Ready to help customers!")

## 🧪 Section 5: Test A2A Communication

Let's test the agent-to-agent communication! We'll ask the Customer Support Agent about movies, and it will communicate with the Movie Catalog Agent via A2A.

### What happens behind the scenes:
1. Customer asks Support Agent a question about a movie
2. Support Agent realizes it needs movie info
3. Support Agent calls the `remote_movie_catalog_agent` (RemoteA2aAgent)
4. ADK sends an A2A protocol request to `http://localhost:8001`
5. Movie Catalog Agent processes the request and responds
6. Support Agent receives the response and continues
7. Customer gets the final answer

All of this happens **transparently** - the Support Agent doesn't need to know it's talking to a remote agent!

### Try More Examples

Let's test a few more scenarios to see A2A communication in action!

In [ ]:
async def test_mia(user_query: str):
    """
    Test the A2A communication between Customer Support Agent and Movie Catalog Agent.

    This function:
    1. Creates a new session for this conversation
    2. Sends the query to the Customer Support Agent
    3. Support Agent communicates with Movie Catalog Agent via A2A
    4. Displays the response

    Args:
        user_query: The question to ask the Customer Support Agent
    """
    # Setup session management (required by ADK)
    session_service = InMemorySessionService()

    # Session identifiers
    app_name = "support_app"
    user_id = "demo_user"
    # Use unique session ID for each test to avoid conflicts
    session_id = f"demo_session_{uuid.uuid4().hex[:8]}"

    # CRITICAL: Create session BEFORE running agent (synchronous, not async!)
    # This pattern matches the deployment notebook exactly
    session = await session_service.create_session(
        app_name=app_name, user_id=user_id, session_id=session_id
    )

    # Create runner for the Customer Support Agent
    # The runner manages the agent execution and session state
    runner = Runner(
        agent=customer_support_agent, app_name=app_name, session_service=session_service
    )

    # Create the user message
    # This follows the same pattern as the deployment notebook
    test_content = types.Content(parts=[types.Part(text=user_query)])

    # Display query
    print(f"\n👤 Customer: {user_query}")
    print(f"\n🎧 Support Agent response:")
    print("-" * 60)

    # Run the agent asynchronously (handles streaming responses and A2A communication)
    async for event in runner.run_async(
        user_id=user_id, session_id=session_id, new_message=test_content
    ):
        # Print final response only (skip intermediate events)
        if event.is_final_response() and event.content:
            for part in event.content.parts:
                if hasattr(part, "text"):
                    print(part.text)

    print("-" * 60)


# Run the test
print("🧪 Testing A2A Communication...\n")
await test_mia("Can you tell me about Sholay? Is it in stock?")

In [ ]:
# Test comparing multiple movies
await test_mia(
    "I'm looking for a movie Sholay. Can you compare Sholay and Dil Chahta Hai for me?"
)

In [ ]:
# Test specific movie inquiry
await test_mia(
    "Do you have the movie The Martian? What's it about?"
)

In [ ]:
# Test specific movie inquiry
await test_mia(
    "Do you have the movie The Martian? Can you recommend other movies like Martian?"
)

---

| Authors |
| --- |
| [Gyanesh Pandey](https://www.linkedin.com/in/gyanesh-pandey/) |